문항 1. 그람 행렬(Gram Matrix)의 의미 스타일 전이에서 '스타일(Style)'을 수치화하기 위해 특징 맵(Feature Map)들 간의 상관관계를 계산하는 **그람 행렬(Gram Matrix)** 을 사용합니다. 그람 행렬이 공간적인 정보(위치)를 무시하고 보존하고자 하는 이미지의 특성은 무엇인지 서술하세요. (예: 물체의 위치 vs 질감/패턴)

답변 작성란:

문항 2. VGG 모델의 역할 신경망 스타일 전이에서는 이미지를 생성하는 네트워크를 처음부터 학습시키는 것이 아니라, 대규모 데이터셋(ImageNet)으로 미리 학습된 VGG19 모델을 가져와 사용합니다. 이때 VGG19 모델의 **가중치(Weight)** 는 학습 도중에 업데이트가 됩니까, 아니면 고정됩니까? 그리고 그 이유는 무엇입니까?

답변 작성란:

가중치 업데이트 여부:

이유:

문항 3. 그람 행렬 (Gram Matrix) 구현 스타일 손실을 계산하기 위한 핵심 함수인 GramMatrix를 구현하세요. 입력 텐서 (batch, channel, height, width)를 (batch, channel, height*width)로 변환한 뒤, 자신과의 행렬 곱을 수행해야 합니다.

In [ ]:
import torch
import torch.nn as nn

class GramMatrix(nn.Module):
    def forward(self, input):
        # 입력 텐서의 크기: (b, c, h, w)
        b, c, h, w = input.size()

        # TODO: 1. 텐서 형태 변환 (Reshape)
        # (b, c, h*w) 형태로 만드세요.
        features = # (코드를 입력하세요)

        # TODO: 2. 그람 행렬 계산 (Matrix Multiplication)
        # features와 features의 전치 행렬(transpose)을 곱하세요.
        # (b, c, c) 크기가 나와야 합니다.
        G = # (코드를 입력하세요)

        # 정규화 (픽셀 수로 나누어 줌)
        return G.div(h * w)

문항 4. 모델 준비 및 파라미터 고정 torchvision에서 제공하는 사전 학습된 vgg19 모델을 불러오고, 스타일 전이 과정에서는 이 모델의 특징 추출 능력만 사용하므로 모든 파라미터의 기울기(Gradient) 계산을 비활성화하는 코드를 작성하세요.

In [ ]:
import torchvision.models as models

# TODO: 사전 학습된 VGG19 모델 불러오기 (features 모듈만 사용)
vgg = # (코드를 입력하세요. 힌트: models.vgg19(pretrained=True).features)

# TODO: 모델의 모든 파라미터에 대해 기울기 계산 비활성화 (requires_grad = False)
for param in vgg.parameters():
    # (코드를 입력하세요)

# GPU 사용 가능 시 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg = vgg.to(device)

문항 5. 스타일 손실 클래스 (Style Loss) 계산된 그람 행렬 간의 차이(MSE)를 계산하는 GramMSELoss 클래스를 완성하세요. target은 스타일 이미지의 그람 행렬 값입니다.

In [ ]:
class GramMSELoss(nn.Module):
    def forward(self, input, target):
        # TODO: 입력(생성된 이미지의 특징)에 대해 그람 행렬 계산
        # 앞서 만든 GramMatrix 클래스 인스턴스 등을 활용하거나 직접 계산
        gram_input = # (코드를 입력하세요. 예: GramMatrix()(input))

        # TODO: Mean Squared Error Loss 계산
        out = # (코드를 입력하세요. 힌트: nn.MSELoss()(gram_input, target))

        return out

문항 6. 총 손실(Total Loss) 계산 및 역전파 스타일 전이의 최종 목표는 콘텐츠 손실과 스타일 손실의 가중치 합을 최소화하는 것입니다. 아래 학습 루프의 일부를 완성하세요.

(style_weight, content_weight가 주어졌다고 가정합니다.)

In [ ]:
# 가정:
# style_losses: 스타일 손실 객체 리스트 (각 층별 Loss)
# content_losses: 콘텐츠 손실 객체 리스트
# style_score, content_score 변수 초기화 필요

total_loss = 0

# TODO: 스타일 손실 합산 (가중치 style_weight 곱하기)
for sl in style_losses:
    style_score += # (코드를 입력하세요: sl.loss)

# TODO: 콘텐츠 손실 합산 (가중치 content_weight 곱하기)
for cl in content_losses:
    content_score += # (코드를 입력하세요: cl.loss)

# TODO: 총 손실 계산 (가중치가 적용된 점수들의 합)
# style_score에는 이미 가중치가 곱해져 있다고 가정하거나, 여기서 곱셈 수행
total_loss = style_score * style_weight + content_score * content_weight

# 역전파
total_loss.backward()